In [29]:
import json, glob, os

base_url = '/static/html/ExRidges/'
default_name = 'not_available.html'

with open('/home/mchin/workspace/sarah/Portal_ExRidges/ExRidges.geojson', "r") as fi:
    data = json.load(fi)
    new_data = {"type": "FeatureCollection"}
    new_data["features"] = [] 
    
    for f in data["features"]:
        feature = {"type": "Feature"}
        feature["geometry"] = f["geometry"]
        feature["properties"] = {}
        feature["properties"]["color"] = [255.0/255,215.0/255,0,1.0]
        feature["properties"]["name"] = f["properties"]["name"]
        feature["properties"]["ID"] = f["properties"]["SEG_ID"]
        
        gid = f["properties"]["SEG_ID"].replace('_', '-')
        full_file_path = glob.glob('/home/mchin/workspace/sarah/Portal_ExRidges/ExRidges_HTML_pages/{}_*.html'.\
                        format(gid))
        if full_file_path:
            base_name = os.path.basename(full_file_path[0])
            print base_name
        else:
            base_name = default_name
        feature["properties"]["width"] = 3.0
        feature["properties"]["URL"] = base_url+base_name
        
        new_data["features"].append(feature)
        
    with open('/home/mchin/workspace/sarah/Portal_ExRidges/ExRidges_new.geojson', "w") as fo:
        fo.write(json.dumps(new_data))

1-1_Aegir_Ridge.html
